In [ ]:
from spacy.lang.en import English
from tqdm.auto import tqdm
from main import page_text
import re
from sentence_transformers import SentenceTransformer
import chromadb
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from huggingface_hub import notebook_login

In [ ]:
nlp = English()
nlp.add_pipe("sentencizer")

for item in tqdm(page_text):
    item['sentences'] = list(nlp(item["text"]).sents)
    item['sentences'] = [str(sentence) for sentence in item['sentences']]
    item['page_sentence_count_spacy'] = len(item['sentences'])

In [ ]:
chunk_size = 10

split_list = lambda lst, n=chunk_size: [lst[i:i + n] for i in range(0, len(lst), n)]

for item in tqdm(page_text):
    sentences = item.get('sentences', [])
    chunks = split_list(sentences)
    item.update({'sentence_chunks': chunks, 'num_chunks': len(chunks)})

for item in tqdm(page_text):
    if 'sentence_chunks' in item:
        for sentence_chunk in item['sentence_chunks']:
            joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
            item['joined_sentence_chunk'] = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)
            item['chunk_token_count'] = len(joined_sentence_chunk) / 4

In [ ]:
model = SentenceTransformer('Alibaba-NLP/gte-base-en-v1.5', trust_remote_code=True).cuda()

client = chromadb.Client()

collection_name = 'pdf_embeddings'
collection = client.create_collection(name=collection_name)

ids = []
embeddings = []

for i, item in enumerate(tqdm(page_text)):
    if 'joined_sentence_chunk' in item:
        chunk = item['joined_sentence_chunk']
        embedding = model.encode(chunk)

        item['embedding'] = embedding
        ids.append(str(i))
        embeddings.append(embedding.tolist())

collection.add(ids=ids, embeddings=embeddings)

print("Data successfully stored in Chroma database.")

In [ ]:
query = "What is Positional Encoding"
query_embedding = model.encode(query).tolist()
results = collection.query(query_embeddings=[query_embedding], n_results=1)
matching_id = results['ids'][0][0]
matching_text = page_text[int(matching_id)]['joined_sentence_chunk']
print(f"Text similar to the query: {matching_text}")

In [ ]:
notebook_login()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=32)

output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)

In [ ]:
prompt_tuning = "Your role is to take the Retrieved data on the query and convert that retrieved data as the query suggest and answer in detail"
input_text = prompt_tuning + query + matching_text
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=200)

output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
output_text = output_text[len(input_text):].strip()

print(output_text)